In [11]:
import sys
!{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!{sys.executable} -m pip install numpy scikit-learn tqdm open3d matplotlib

# Импорты
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import StepLR
import numpy as np
import os
from tqdm import tqdm
from collections import defaultdict
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import open3d as o3d
from collections import defaultdict, Counter 

Looking in indexes: https://download.pytorch.org/whl/cu121



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 🔁 ЗАМЕНИТЕ НА СВОЙ ПУТЬ!
S3DIS_ROOT = "C:/Users/anast/ml/CVin3D/lab4/task2/Stanford3dDataset"  # ← ваш путь к Area_1, Area_2 и т.д.
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

CLASS_NAMES = [
    'ceiling', 'floor', 'wall', 'beam', 'column',
    'window', 'door', 'table', 'chair', 'sofa',
    'bookcase', 'board', 'clutter', 'stairs'
]
NUM_CLASSES = len(CLASS_NAMES)
print(f"Всего классов: {NUM_CLASSES}")

Всего классов: 14


In [4]:
def process_room_from_array(data: np.ndarray) -> np.ndarray:
    """Вход: (N, 7) → x,y,z,r,g,b,label → выход: (N, 6+1) нормализованные xyz+rgb+label"""
    if data.shape[1] < 7:
        raise ValueError(f"Ожидалось ≥7 столбцов, получено {data.shape[1]}")
    xyz = data[:, :3]
    rgb = data[:, 3:6]
    labels = data[:, 6].astype(np.int64)

    rgb = np.clip(rgb / 255.0, 0.0, 1.0)
    xyz -= np.mean(xyz, axis=0)
    scale = np.max(np.abs(xyz)) + 1e-8
    xyz /= scale

    return np.hstack([xyz, rgb, labels.reshape(-1, 1)])


def process_area_with_meta(area_dir: str, area_id: int) -> np.ndarray:
    """Возвращает массив (N, 9): x,y,z,r,g,b,label,area_id,room_id_hash"""
    all_points = []
    
    for room_name in sorted(os.listdir(area_dir)):
        room_path = os.path.join(area_dir, room_name)
        if not os.path.isdir(room_path):
            continue

        ann_path = os.path.join(room_path, 'Annotations')
        room_data = []
        if os.path.isdir(ann_path):
            obj_files = [f for f in os.listdir(ann_path) if f.endswith('.txt')]
            for obj_file in obj_files:
                obj_full = os.path.join(ann_path, obj_file)
                obj_name = os.path.splitext(obj_file)[0].split('_')[0]
                try:
                    label_id = CLASS_NAMES.index(obj_name)
                except ValueError:
                    # print(f"⚠️ Пропускаем неизвестный класс: {obj_name}")
                    continue
                try:
                    obj_pts = np.genfromtxt(obj_full, delimiter=' ', dtype=np.float32)
                    if obj_pts.ndim == 1:
                        obj_pts = obj_pts.reshape(1, -1)
                    if obj_pts.shape[1] == 6:
                        obj_pts = np.hstack([obj_pts, np.full((obj_pts.shape[0], 1), label_id)])
                    elif obj_pts.shape[1] >= 7:
                        obj_pts = obj_pts[:, :7]  # берём только первые 7
                        obj_pts[:, 6] = label_id
                    room_data.append(obj_pts)
                except Exception as e:
                    print(f"❌ Ошибка {obj_full}: {e}")
                    continue

            if room_data:
                room_points = np.vstack(room_data)
                room_processed = process_room_from_array(room_points)
                # Добавляем meta: area_id, room_hash
                room_hash = abs(hash(room_name)) % 100000
                meta = np.full((room_processed.shape[0], 2), [area_id, room_hash])
                room_with_meta = np.hstack([room_processed, meta])
                all_points.append(room_with_meta)
        else:
            # Если нет Annotations — ищем room*.txt
            room_txts = [f for f in os.listdir(room_path) if f.startswith('room') and f.endswith('.txt')]
            for txt in room_txts:
                full_txt = os.path.join(room_path, txt)
                try:
                    data_raw = np.genfromtxt(full_txt, delimiter=' ', dtype=np.float32)
                    if data_raw.shape[1] >= 7:
                        data_proc = process_room_from_array(data_raw[:, :7])
                        room_hash = abs(hash(txt)) % 100000
                        meta = np.full((data_proc.shape[0], 2), [area_id, room_hash])
                        all_points.append(np.hstack([data_proc, meta]))
                except Exception as e:
                    print(f"❌ Ошибка обработки {full_txt}: {e}")

    if not all_points:
        print(f"⚠️ Area_{area_id}: нет данных")
        return np.empty((0, 9), dtype=np.float32)

    return np.vstack(all_points)

In [14]:
area_files = []

for i in range(1, 7):
    area_path = os.path.join(S3DIS_ROOT, f"Area_{i}")
    if not os.path.isdir(area_path):
        print(f" Пропущен Area_{i}: папка не найдена")
        continue

    print(f"Обрабатываю Area_{i}...")
    try:
        area_data = process_area_with_meta(area_path, area_id=i)  # (N, 9), float64
        print(f"  → {area_data.shape[0]:,} точек, {area_data.dtype}")
        
        area_data = area_data.astype(np.float32)
        area_file = os.path.join(OUTPUT_DIR, f"area_{i}.npy")
        np.save(area_file, area_data)
        area_files.append(area_file)
        print(f"   Сохранено: {area_file} | {os.path.getsize(area_file) / 1e6:.1f} MB")
        
        del area_data
    except MemoryError as e:
        print(f" MemoryError в Area_{i}: {e}")
        raise
    except Exception as e:
        print(f" Ошибка обработки Area_{i}: {e}")
        continue

print(f"\n Обработано {len(area_files)} Area.")

Обрабатываю Area_1...
❌ Ошибка C:/Users/anast/ml/CVin3D/lab4/task2/Stanford3dDataset\Area_1\office_22\Annotations\clutter_16.txt: Unable to allocate 1.08 MiB for an array with shape (20171, 7) and data type float64
 MemoryError в Area_1: Unable to allocate 45.7 MiB for an array with shape (856367, 7) and data type float64


MemoryError: Unable to allocate 45.7 MiB for an array with shape (856367, 7) and data type float64

In [15]:
def load_room_txt(file_path):
    """Загружает комнату из .txt, возвращает (N, 7): xyzrgb + class_id"""
    data = np.loadtxt(file_path)  # (N, 7)
    # Нормализуем цвет в [0,1]
    data[:, 3:6] = data[:, 3:6] / 255.0
    return data.astype(np.float32)  # экономим память: float32!

def room_to_blocks(data, block_size=1.5, stride=1.0, max_points=4096, min_points=1024):
    """Режет облако на блоки фикс. размера (в метрах), возвращает список блоков"""
    blocks = []
    labels = []

    xyz = data[:, :3]
    mins = np.min(xyz, axis=0)
    maxs = np.max(xyz, axis=0)

    # Сетка блоков по X и Y (Z — вся высота комнаты)
    x_starts = np.arange(mins[0], maxs[0], stride)
    y_starts = np.arange(mins[1], maxs[1], stride)

    for x in x_starts:
        for y in y_starts:
            x_mask = (xyz[:, 0] >= x) & (xyz[:, 0] < x + block_size)
            y_mask = (xyz[:, 1] >= y) & (xyz[:, 1] < y + block_size)
            mask = x_mask & y_mask
            block = data[mask]
            if len(block) < min_points:
                continue
            # Сэмплируем до max_points
            if len(block) > max_points:
                idx = np.random.choice(len(block), max_points, replace=False)
                block = block[idx]
            else:
                # дополняем до max_points (реже, но безопасно)
                idx = np.random.choice(len(block), max_points, replace=True)
                block = block[idx]
            blocks.append(block[:, :6])   # xyzrgb
            labels.append(block[:, 6].astype(np.int64))  # class

    return blocks, labels

class S3DISBlockDataset(Dataset):
    def __init__(self, area_path, room_limit=None, block_size=1.5, max_points=4096):
        self.blocks = []
        self.labels = []
        room_dirs = sorted([d for d in os.listdir(area_path) if os.path.isdir(os.path.join(area_path, d))])
        
        if room_limit:
            room_dirs = room_dirs[:room_limit]  # ⚠️ ОГРАНИЧИВАЕМ ЧИСЛО КОМНАТ!
        
        print(f"Загрузка {len(room_dirs)} комнат из {area_path}...")
        for room in room_dirs:
            room_path = os.path.join(area_path, room, "Annotations")
            if not os.path.exists(room_path):
                continue
            # Собираем все объекты комнаты (стены, пол и т.д.)
            obj_files = [f for f in os.listdir(room_path) if f.endswith('.txt')]
            room_data = []
            for obj_file in obj_files:
                obj_path = os.path.join(room_path, obj_file)
                try:
                    obj_data = np.loadtxt(obj_path)
                    # Добавляем class_id из имени файла, например: "wall_1.txt" → class 2
                    cls_name = obj_file.split('_')[0].lower()
                    # Простое отображение (для демо — 13 классов не критично):
                    class_map = {'ceiling':0, 'floor':1, 'wall':2, 'beam':3, 'column':4,
                                 'window':5, 'door':6, 'table':7, 'chair':8, 'sofa':9,
                                 'bookcase':10, 'board':11, 'clutter':12}
                    cls_id = class_map.get(cls_name, 12)
                    obj_data = np.column_stack([obj_data, np.full(len(obj_data), cls_id)])
                    room_data.append(obj_data)
                except Exception as e:
                    print(f"⚠️ Пропущен файл {obj_file}: {e}")
                    continue
            if not room_data:
                continue
            room_data = np.vstack(room_data).astype(np.float32)
            # Режем на блоки
            blocks, labels = room_to_blocks(room_data, block_size=block_size, max_points=max_points)
            self.blocks.extend(blocks)
            self.labels.extend(labels)
            if len(self.blocks) > 500:  # ⚠️ ОГРАНИЧИВАЕМ ЧИСЛО БЛОКОВ!
                break

        print(f"✅ Загружено {len(self.blocks)} блоков.")

    def __len__(self):
        return len(self.blocks)

    def __getitem__(self, idx):
        points = torch.from_numpy(self.blocks[idx])
        labels = torch.from_numpy(self.labels[idx])
        return points, labels

In [ ]:
total_points = 0
for f in area_files:
    with open(f, "rb") as fh:
        import numpy.lib.format as npfmt
        version = npfmt.read_magic(fh)
        header = npfmt._read_array_header(fh, version)
        shape = header[1]
        total_points += shape[0]

print(f"Общее число точек: {total_points:,}")

NPY_PATH = os.path.join(OUTPUT_DIR, "s3dis_full.npy")
with open(NPY_PATH, "wb") as f_out:
    import struct
    import numpy as np

    header_dict = {
        'descr': '<f4',  # float32, little-endian
        'fortran_order': False,
        'shape': (total_points, 9)
    }
    header_str = f"{{'descr': '{header_dict['descr']}', 'fortran_order': {header_dict['fortran_order']}, 'shape': {header_dict['shape']}, }}"
    header_len = len(header_str) + 1  # +1 for \n
    padding = (64 - (10 + header_len) % 64) % 64
    header = f"\x93NUMPY\x01\x00{''.ljust(2, '\x00')}" \
             + struct.pack('<H', header_len + padding) \
             + header_str + ' ' * padding + '\n'

    f_out.write(header.encode('latin1'))

    written = 0
    for f_path in area_files:
        print(f"Добавляю {os.path.basename(f_path)}...")
        arr = np.load(f_path, mmap_mode='r')  
        arr.tofile(f_out)
        written += arr.shape[0]
        del arr  

print(f" Итоговый файл создан: {NPY_PATH}")
print(f"   Размер: {os.path.getsize(NPY_PATH) / 1e9:.2f} GB")

In [16]:
# 🔁 Укажи ПРАВИЛЬНЫЙ путь к S3DIS!
S3DIS_ROOT = "C:/Users/anast/ml/CVin3D/lab4/task2/Stanford3dDataset"  # ← ЗАМЕНИ НА СВОЙ!

area1_path = os.path.join(S3DIS_ROOT, "Area_1")

# Загружаем ТОЛЬКО 2 комнаты, не более 200 блоков
try:
    dataset = S3DISBlockDataset(area1_path, room_limit=2)  # ⬅️ КЛЮЧ: room_limit=2
except Exception as e:
    print("❌ Ошибка:", e)
    print("\n❗ Если путь неверный — загрузим демо-данные (из памяти)...")
    # fallback: генерация 50 блоков игрушечных данных (как в прошлом ответе)
    from sklearn.datasets import make_blobs
    blocks, labels = [], []
    for _ in range(50):
        pts, lbls = generate_room_points(num_points=4096)
        blocks.append(pts)
        labels.append(lbls)
    dataset = ToyPointCloudDataset(np.stack(blocks), np.stack(labels))

print("Размер блока:", dataset[0][0].shape)  # должно быть (4096, 6)

Загрузка 2 комнат из C:/Users/anast/ml/CVin3D/lab4/task2/Stanford3dDataset\Area_1...
✅ Загружено 62 блоков.
Размер блока: torch.Size([4096, 6])


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- Вспомогательные функции (без изменений, работают) ---
def square_distance(src, dst):
    return torch.sum((src[:, :, None, :] - dst[:, None, :, :]) ** 2, dim=-1)

def farthest_point_sample(xyz, npoint):
    device = xyz.device
    B, N, _ = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long, device=device)
    distance = torch.ones(B, N, device=device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long, device=device)
    batch_indices = torch.arange(B, dtype=torch.long, device=device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = torch.max(distance, -1)[1]
    return centroids

def index_points(points, idx):
    raw_shape = idx.shape
    idx = idx.reshape(raw_shape[0], -1)
    res = torch.gather(points, 1, idx.unsqueeze(-1).expand(-1, -1, points.size(-1)))
    return res.reshape(*raw_shape, -1)

def query_ball_point(radius, nsample, xyz, new_xyz):
    device = xyz.device
    B, N, _ = xyz.shape
    S = new_xyz.shape[1]
    K = nsample
    group_idx = torch.arange(N, dtype=torch.long, device=device).view(1, 1, N).repeat(B, S, 1)
    sqrdists = square_distance(new_xyz, xyz)
    group_idx[sqrdists > radius ** 2] = N
    group_idx = group_idx.sort(dim=-1)[0][:, :, :K]
    group_first = group_idx[:, :, 0].view(B, S, 1).repeat(1, 1, K)
    mask = group_idx == N
    group_idx[mask] = group_first[mask]
    return group_idx

# --- Исправленный Set Abstraction (работает и с points=None, и с points) ---
class PointNetSetAbstraction(nn.Module):
    def __init__(self, npoint, radius, nsample, in_channel, mlp):
        super().__init__()
        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        
        # ВАЖНО: ожидаемый вход — (C_local + 3), где C_local = in_channel (rgb/feat), 3 = dxyz
        last_channel = in_channel + 3
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv2d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm2d(out_channel))
            last_channel = out_channel

    def forward(self, xyz, points):
        """
        xyz: (B, N, 3)
        points: (B, N, C) — может быть None (тогда C=0), но лучше передавать хотя бы rgb
        """
        B, N, _ = xyz.shape
        S = self.npoint

        fps_idx = farthest_point_sample(xyz, S)   # (B, S)
        new_xyz = index_points(xyz, fps_idx)      # (B, S, 3)
        idx = query_ball_point(self.radius, self.nsample, xyz, new_xyz)  # (B, S, K)
        grouped_xyz = index_points(xyz, idx)      # (B, S, K, 3)
        grouped_xyz_norm = grouped_xyz - new_xyz.view(B, S, 1, 3)  # (B, S, K, 3)

        if points is not None:
            grouped_points = index_points(points, idx)     # (B, S, K, C)
            new_points = torch.cat([grouped_xyz_norm, grouped_points], dim=-1)  # (B, S, K, C+3)
        else:
            new_points = grouped_xyz_norm  # (B, S, K, 3) → тогда in_channel должен быть 0!

        new_points = new_points.permute(0, 3, 2, 1)  # (B, C+3, K, S)
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points = F.relu(bn(conv(new_points)))
        new_points = torch.max(new_points, 2)[0]  # (B, C_out, S)
        new_points = new_points.permute(0, 2, 1)  # (B, S, C_out)
        return new_xyz, new_points

class PointNetFeaturePropagation(nn.Module):
    def __init__(self, in_channel, mlp):
        super().__init__()
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv1d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm1d(out_channel))
            last_channel = out_channel

    def forward(self, xyz1, xyz2, points1, points2):
        B, N, _ = xyz1.shape
        _, S, _ = xyz2.shape

        if S == 1:
            interpolated_points = points2.repeat(1, N, 1)
        else:
            dists = square_distance(xyz1, xyz2)
            dists, idx = dists.sort(dim=-1)
            dists, idx = dists[:, :, :3], idx[:, :, :3]
            dist_recip = 1.0 / (dists + 1e-8)
            norm = torch.sum(dist_recip, dim=2, keepdim=True)
            weight = dist_recip / norm
            interpolated_points = torch.sum(index_points(points2, idx) * weight.unsqueeze(-1), dim=2)

        if points1 is not None:
            new_points = torch.cat([points1, interpolated_points], dim=-1)
        else:
            new_points = interpolated_points

        new_points = new_points.permute(0, 2, 1)
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points = F.relu(bn(conv(new_points)))
        return new_points.permute(0, 2, 1)

# ✅ ФИНАЛЬНАЯ РАБОЧАЯ МОДЕЛЬ
class MiniPointNet2Seg(nn.Module):
    def __init__(self, num_classes=13):
        super().__init__()
        # Первый уровень: вход — xyz (3) + rgb (3) = 6 → поэтому in_channel=3 (rgb), +3 (dxyz) = 6
        self.sa1 = PointNetSetAbstraction(
            npoint=512,
            radius=0.2,
            nsample=32,
            in_channel=3,   # ← это rgb (или любые локальные признаки, кроме xyz)
            mlp=[32, 64]
        )
        self.fp1 = PointNetFeaturePropagation(in_channel=64 + 3, mlp=[128, 128])  # 64 (sa1) + 3 (rgb)
        self.conv1 = nn.Conv1d(128, 128, 1)
        self.bn1 = nn.BatchNorm1d(128)
        self.drop1 = nn.Dropout(0.5)
        self.conv2 = nn.Conv1d(128, num_classes, 1)

    def forward(self, xyzrgb):
        # xyzrgb: (B, N, 6)
        B, N, _ = xyzrgb.shape
        xyz = xyzrgb[:, :, :3]    # (B, N, 3)
        rgb = xyzrgb[:, :, 3:]    # (B, N, 3) ← передаём как points!

        # SA: xyz + rgb → признаки
        new_xyz, feat = self.sa1(xyz, rgb)  # feat: (B, 512, 64)

        # FP: upsample до N, используем rgb как локальный признак на входе
        feat_up = self.fp1(xyz, new_xyz, rgb, feat)  # (B, N, 128)

        # Seg head
        x = feat_up.permute(0, 2, 1)  # (B, 128, N)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.drop1(x)
        x = self.conv2(x)  # (B, C, N)
        return x.permute(0, 2, 1)  # (B, N, C)

In [ ]:
# Проверка формы
device = torch.device("cpu")
model = MiniPointNet2Seg(num_classes=13).to(device)
dummy_input = torch.randn(2, 4096, 6)  # batch=2, N=4096, xyzrgb
with torch.no_grad():
    out = model(dummy_input)
print(" Форма выхода:", out.shape)  # должно быть: torch.Size([2, 4096, 13])

✅ Форма выхода: torch.Size([2, 4096, 13])


In [ ]:
device = torch.device("cpu")
model = MiniPointNet2Seg(num_classes=13).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(" Запуск обучения (без .view — только .reshape)...")
model.train()
for epoch in range(1, 3):
    total_loss = 0.0
    for i, (points, labels) in enumerate(train_loader):
        points, labels = points.to(device), labels.to(device)
        
        optimizer.zero_grad()
        out = model(points)  # (B, N, 13)
        
        # 🔑 КЛЮЧЕВОЕ ИЗМЕНЕНИЕ: .reshape вместо .view
        loss = criterion(out.reshape(-1, 13), labels.reshape(-1))
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        if i == 2:  # 3 шага — для проверки
            break
    
    print(f"Epoch {epoch} | Avg Loss (3 steps): {total_loss / 3:.4f}")

print("🔥 УСПЕХ! Модель обучается на CPU.")

✅ Запуск обучения (без .view — только .reshape)...
Epoch 1 | Avg Loss (3 steps): 2.4620
Epoch 2 | Avg Loss (3 steps): 1.9567
🔥 УСПЕХ! Модель обучается на CPU.


In [ ]:
class S3DISAreaSplitDataset(Dataset):
    def __init__(self, npy_path, split='train', num_points=4096, augment=True):
        self.data = np.load(npy_path)  # (N, 9)
        self.num_points = num_points
        self.augment = augment and (split == 'train')
        
        area_ids = self.data[:, 7].astype(int)
        if split == 'train':
            mask = area_ids <= 5  
        elif split == 'val':
            mask = area_ids == 5  
        elif split == 'test':
            mask = area_ids == 6  
        else:
            raise ValueError("split = 'train'/'val'/'test'")
        
        self.points = self.data[mask][:, :7]  # x,y,z,r,g,b,label
        print(f"{split.capitalize()}: {self.points.shape[0]:,} точек")

    def __len__(self):
        return 1000

    def __getitem__(self, idx):
        N = self.points.shape[0]
        choice = np.random.choice(N, self.num_points, replace=N < self.num_points)
        batch = self.points[choice]  

        xyz = batch[:, :3].copy()
        rgb = batch[:, 3:6].copy()
        label = batch[:, 6].astype(np.int64)

        if self.augment:
            angle = np.random.uniform(0, 2 * np.pi)
            cos_a, sin_a = np.cos(angle), np.sin(angle)
            rot = np.array([[cos_a, -sin_a, 0],
                            [sin_a,  cos_a, 0],
                            [0,      0,     1]])
            xyz = xyz @ rot
            xyz += np.clip(0.01 * np.random.randn(*xyz.shape), -0.05, 0.05)

        xyz -= np.mean(xyz, axis=0)
        scale = np.max(np.linalg.norm(xyz, axis=1)) + 1e-8
        xyz /= scale

        pts = np.concatenate([xyz, rgb], axis=1).astype(np.float32)  # (P, 6)
        return torch.from_numpy(pts), torch.from_numpy(label).long()

In [ ]:
NPY_PATH = os.path.join(OUTPUT_DIR, "s3dis_full.npy")

NUM_POINTS = 1024   
BATCH_SIZE = 2      
NUM_WORKERS = 0     

train_ds = S3DISAreaSplitDataset(NPY_PATH, split='train', num_points=NUM_POINTS)
val_ds = S3DISAreaSplitDataset(NPY_PATH, split='val', num_points=NUM_POINTS)
test_ds = S3DISAreaSplitDataset(NPY_PATH, split='test', num_points=NUM_POINTS)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print(" DataLoader готов.")

MemoryError: Unable to allocate 18.3 GiB for an array with shape (2461918383,) and data type float64

In [ ]:
def square_distance(src, dst):
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.transpose(2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist

def index_points(points, idx):
    device = points.device
    B = points.shape[0]
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1
    batch_indices = torch.arange(B, dtype=torch.long, device=device).view(view_shape).repeat(repeat_shape)
    new_points = points[batch_indices, idx, :]
    return new_points

def farthest_point_sample(xyz, npoint):
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long, device=device)
    distance = torch.ones(B, N, device=device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long, device=device)
    batch_indices = torch.arange(B, dtype=torch.long, device=device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = torch.max(distance, -1)[1]
    return centroids

def query_ball_point(radius, nsample, xyz, new_xyz):
    device = xyz.device
    B, N, _ = xyz.shape
    _, S, _ = new_xyz.shape
    group_idx = torch.arange(N, dtype=torch.long).to(device).view(1, 1, N).repeat([B, S, 1])
    sqrdists = square_distance(new_xyz, xyz)
    group_idx[sqrdists > radius ** 2] = N
    group_idx = group_idx.sort(dim=-1)[0][:, :, :nsample]
    group_first = group_idx[:, :, 0].view(B, S, 1).repeat([1, 1, nsample])
    mask = group_idx == N
    group_idx[mask] = group_first[mask]
    return group_idx

def sample_and_group(npoint, radius, nsample, xyz, points):
    B, N, C = xyz.shape
    S = npoint
    fps_idx = farthest_point_sample(xyz, npoint)
    new_xyz = index_points(xyz, fps_idx)
    idx = query_ball_point(radius, nsample, xyz, new_xyz)
    grouped_xyz = index_points(xyz, idx)
    grouped_xyz_norm = grouped_xyz - new_xyz.view(B, S, 1, 3)
    if points is not None:
        grouped_points = index_points(points, idx)
        new_points = torch.cat([grouped_xyz_norm, grouped_points], dim=-1)
    else:
        new_points = grouped_xyz_norm
    return new_xyz, new_points

class PointNetSetAbstraction(nn.Module):
    def __init__(self, npoint, radius, nsample, in_channel, mlp, group_all=False):
        super().__init__()
        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv2d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm2d(out_channel))
            last_channel = out_channel
        self.group_all = group_all

    def forward(self, xyz, points):
        if self.group_all:
            new_xyz = xyz.mean(dim=1, keepdim=True)
            grouped_points = points.unsqueeze(1)
            grouped_xyz = xyz.unsqueeze(1) - new_xyz.unsqueeze(2)
            if points is not None:
                new_points = torch.cat([grouped_xyz, grouped_points], dim=-1)
            else:
                new_points = grouped_xyz
        else:
            new_xyz, new_points = sample_and_group(self.npoint, self.radius, self.nsample, xyz, points)

        new_points = new_points.permute(0, 3, 2, 1)
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points = F.relu(bn(conv(new_points)))
        new_points = torch.max(new_points, 2)[0]
        new_points = new_points.permute(0, 2, 1)
        return new_xyz, new_points

class PointNetFeaturePropagation(nn.Module):
    def __init__(self, in_channel, mlp):
        super().__init__()
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv1d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm1d(out_channel))
            last_channel = out_channel

    def forward(self, xyz1, xyz2, points1, points2):
        B, N, _ = xyz1.shape
        _, S, _ = xyz2.shape

        if S == 1:
            interpolated_points = points2.repeat(1, N, 1)
        else:
            dists = square_distance(xyz1, xyz2)
            dists, idx = dists.sort(dim=-1)
            dists, idx = dists[:, :, :3], idx[:, :, :3]
            dist_recip = 1.0 / (dists + 1e-8)
            norm = torch.sum(dist_recip, dim=2, keepdim=True)
            weight = dist_recip / norm
            interpolated_points = torch.sum(index_points(points2, idx) * weight.unsqueeze(-1), dim=2)

        if points1 is not None:
            new_points = torch.cat([points1, interpolated_points], dim=-1)
        else:
            new_points = interpolated_points

        new_points = new_points.permute(0, 2, 1)
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points = F.relu(bn(conv(new_points)))
        return new_points.permute(0, 2, 1)

class PointNet2Seg(nn.Module):
    def __init__(self, num_classes=14): 
        super().__init__()
        self.sa1 = PointNetSetAbstraction(512, 0.2, 32, 6, [32, 32, 64])
        self.sa2 = PointNetSetAbstraction(128, 0.4, 64, 64 + 3, [64, 64, 128])
        self.sa3 = PointNetSetAbstraction(None, None, None, 128 + 3, [128, 256, 512], group_all=True)

        self.fp3 = PointNetFeaturePropagation(512 + 128, [256, 256])
        self.fp2 = PointNetFeaturePropagation(256 + 64, [256, 128])
        self.fp1 = PointNetFeaturePropagation(128 + 6, [128, 128, 128])

        self.conv1 = nn.Conv1d(128, 128, 1)
        self.bn1 = nn.BatchNorm1d(128)
        self.drop1 = nn.Dropout(0.5)
        self.conv2 = nn.Conv1d(128, num_classes, 1)

    def forward(self, xyz):
        l0_xyz = xyz[:, :, :3]
        l0_points = xyz[:, :, 3:]

        l1_xyz, l1_points = self.sa1(l0_xyz, l0_points)
        l2_xyz, l2_points = self.sa2(l1_xyz, l1_points)
        l3_xyz, l3_points = self.sa3(l2_xyz, l2_points)

        l2_points = self.fp3(l2_xyz, l3_xyz, l2_points, l3_points)
        l1_points = self.fp2(l1_xyz, l2_xyz, l1_points, l2_points)
        l0_points = self.fp1(l0_xyz, l1_xyz, l0_points, l1_points)

        x = l0_points.permute(0, 2, 1)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.drop1(x)
        x = self.conv2(x)
        return x.permute(0, 2, 1)  # (B, N, C)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

model = PointNet2Seg(num_classes=NUM_CLASSES).to(device)
print("Модель загружена на", next(model.parameters()).device)

train_labels = train_ds.points[:, 6].astype(int)
class_counts = np.bincount(train_labels, minlength=NUM_CLASSES)
class_weights = 1.0 / (class_counts + 1e-8)
class_weights = torch.from_numpy(class_weights).float().to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)

def compute_metrics(pred, target, num_classes=NUM_CLASSES):
    pred = pred.view(-1)
    target = target.view(-1)
    cm = confusion_matrix(target.cpu().numpy(), pred.cpu().numpy(), labels=np.arange(num_classes))
    oa = np.diag(cm).sum() / cm.sum()
    iou = np.diag(cm) / (cm.sum(axis=1) + cm.sum(axis=0) - np.diag(cm) + 1e-6)
    miou = np.nanmean(iou)
    return oa, miou

def validate(model, loader, criterion, device):
    model.eval()
    total_loss, all_preds, all_targets = 0.0, [], []
    with torch.no_grad():
        for pts, lbl in loader:
            pts, lbl = pts.to(device), lbl.to(device)
            logits = model(pts)
            loss = criterion(logits.permute(0, 2, 1), lbl)
            total_loss += loss.item()
            all_preds.append(logits.argmax(-1))
            all_targets.append(lbl)
    avg_loss = total_loss / len(loader)
    preds = torch.cat(all_preds)
    targets = torch.cat(all_targets)
    oa, miou = compute_metrics(preds, targets)
    return avg_loss, oa, miou

EPOCHS = 10  # ← для CPU достаточно 5–10 эпох для проверки
train_losses, val_losses, val_oas, val_mious = [], [], [], []
best_miou = 0.0

print(" Начинаем обучение...")
for epoch in range(1, EPOCHS + 1):
    model.train()
    epoch_loss = 0.0
    for pts, lbl in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        pts, lbl = pts.to(device), lbl.to(device)
        optimizer.zero_grad()
        logits = model(pts)
        loss = criterion(logits.permute(0, 2, 1), lbl)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    val_loss, val_oa, val_miou = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_oas.append(val_oa)
    val_mious.append(val_miou)
    
    scheduler.step()
    
    print(f"Epoch {epoch:2d} | Train Loss: {avg_train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | OA: {val_oa:.3f} | mIoU: {val_miou:.3f}")
    
    if val_miou > best_miou:
        best_miou = val_miou
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "best_pointnet2.pth"))
        print("   → Сохранена лучшая модель!")

print(" Обучение завершено.")

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(val_oas, label='OA')
plt.plot(val_mious, label='mIoU')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.title('Metrics')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "training_curves.png"), dpi=150)
plt.show()

model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "best_pointnet2.pth"), weights_only=True))
test_loss, test_oa, test_miou = validate(test_loader)
print(f"\n Результаты на TEST (Area_6):\n  OA = {test_oa:.4f}\n  mIoU = {test_miou:.4f}")

In [ ]:
1